In [1]:
# Import Libraries

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, recall_score

In [2]:
# Load and Inspect the Data (replace with actual path)
data = pd.read_csv('C:/Users/CAMARA NADEGE/Desktop/biomedical data analysis/heart_disease_1.csv')
data

,age,sex,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,heart_status
0,63,0.0,0.0,145.00,233.0,1.0,2.0,150.00,NaN,2.300,2.0,0.0,1,0.0
1,67,0.0,3.0,160.00,286.0,0.0,2.0,108.00,1.0,1.500,1.0,3.0,0,2.0
2,67,0.0,3.0,120.00,229.0,0.0,2.0,129.00,1.0,2.600,1.0,2.0,2,1.0
3,37,0.0,2.0,130.00,250.0,0.0,0.0,187.00,0.0,3.500,2.0,0.0,0,0.0
4,41,1.0,1.0,130.00,204.0,0.0,2.0,172.00,0.0,1.400,0.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
914,54,1.0,3.0,127.00,333.0,1.0,1.0,154.00,0.0,0.000,1.0,0.0,0,1.0
915,62,0.0,0.0,143.06,139.0,NaN,1.0,119.94,1.0,2.042,1.0,NaN,2,0.0
916,55,NaN,3.0,122.00,223.0,1.0,1.0,100.00,0.0,0.000,1.0,1.0,1,2.0
917,58,0.0,3.0,150.81,385.0,1.0,2.0,110.63,1.0,2.181,1.0,0.0,2,NaN


In [14]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Rearranged patient data
patients = {
    "Patient CL": [64, 1, 3, 110, 211, 0, 0, 144, 1, 1.8, 1, 0, 2],
    "Patient CK": [63, 1, 3, 140, 463, 0, 2, 104, 0, 4, 1, 3, 2],
    "Patient CT": [54, 1, 0, 98, 306, 1, 0, 128, 1, 0, 1, 1, 0],
    "Patient CJ": [39, 0, 1, 120, 260, 0, 1, 202, 0, 0.9, 2, 0, 2],
    "Patient CD": [77, 1, 1, 356, 145, 0, 1, 143, 0, 0, 1, 0, 2],
    "Patient CZ": [45, 0, 3, 102, 333, 0, 2, 117, 1, 0, 1, 3, 1],
    "Patient CA": [22, 1, 3, 108, 194, 1, 0, 136, 0, 1.9, 1, 3, 0],
    "Patient CF": [51, 0, 2, 190, 0, 0, 0, 92, 0, 2.4, 1, 2, 0],
    "Patient CV": [48, 0, 2, 140, 141.15, 0, 1, 93.34, 1, 1.5, 1, 3, 2],
    "Patient CM": [56, 0, 1, 135.82, 142, 1, 0, 226.17, 1, 2.545, 1, 2, 2]
}

# Create a DataFrame from the patient data
patient_df = pd.DataFrame.from_dict(patients, orient='index', columns=[
    'age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalch', 
    'exang', 'oldpeak', 'slope', 'ca', 'thal'
])

# Generate random binary 'heart_status' for demonstration purposes
np.random.seed(42)
patient_df['heart_status'] = np.random.randint(0, 2, size=patient_df.shape[0])

# Split the data into features and target
X = patient_df.drop(columns=['heart_status'])
y = patient_df['heart_status']

# Apply SMOTE to the training set with k_neighbors=1
smote = SMOTE(random_state=42, k_neighbors=1)
X_smote, y_smote = smote.fit_resample(X, y)

# Check for NaN/Inf after SMOTE
X_smote = np.nan_to_num(X_smote, nan=0, posinf=0, neginf=0)

# Polynomial Feature Expansion to include interaction terms
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_poly = poly.fit_transform(X_smote)

# Check for NaN/Inf after Polynomial Features
X_poly = np.nan_to_num(X_poly, nan=0, posinf=0, neginf=0)

# Scale the features after SMOTE
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_poly)

# Check for NaN/Inf after Scaling
X_scaled = np.nan_to_num(X_scaled, nan=0, posinf=0, neginf=0)

# Define base models with hyperparameter tuning
rf_model = RandomForestClassifier(random_state=42)
rf_params = {'n_estimators': [200, 300, 400], 'max_depth': [6, 8, 10], 'min_samples_split': [2, 5]}
rf_search = GridSearchCV(rf_model, rf_params, cv=3, scoring='accuracy', n_jobs=-1)
rf_search.fit(X_scaled, y_smote)

gb_model = GradientBoostingClassifier(random_state=42)
gb_params = {'n_estimators': [200, 300], 'max_depth': [3, 4, 5], 'learning_rate': [0.01, 0.05, 0.1]}
gb_search = GridSearchCV(gb_model, gb_params, cv=3, scoring='accuracy', n_jobs=-1)
gb_search.fit(X_scaled, y_smote)

lr_model = LogisticRegression(random_state=42, max_iter=500, solver='liblinear')

# Adding Decision Tree as a new model
dt_model = DecisionTreeClassifier(random_state=42)
dt_params = {'max_depth': [4, 6, 8], 'min_samples_split': [2, 5]}
dt_search = GridSearchCV(dt_model, dt_params, cv=3, scoring='accuracy', n_jobs=-1)
dt_search.fit(X_scaled, y_smote)

# Ensemble Model using Voting Classifier with hard voting for potentially higher accuracy
ensemble_model = VotingClassifier(
    estimators=[
        ('rf', rf_search.best_estimator_), 
        ('gb', gb_search.best_estimator_), 
        ('lr', lr_model),
        ('dt', dt_search.best_estimator_)
    ],
    voting='hard'
)

# Train the ensemble model on the SMOTE-enhanced data
ensemble_model.fit(X_scaled, y_smote)

# Use cross-validation to evaluate model accuracy
cv_scores = cross_val_score(ensemble_model, X_scaled, y_smote, cv=5, scoring='accuracy')
print(f"Cross-Validated Model Accuracy: {cv_scores.mean():.4f}")

# Prepare test data for prediction (ensure no feature names in input to `transform`)
X_test_poly = poly.transform(X.values)
X_test_scaled = scaler.transform(X_test_poly)
X_test_scaled = np.nan_to_num(X_test_scaled, nan=0, posinf=0, neginf=0)  # Handle NaNs/Infs in test data

predictions = ensemble_model.predict(X_test_scaled)

# Display predictions
patient_df['predicted_heart_status'] = predictions
print("\nPatient Predictions:")
print(patient_df[['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalch', 
                  'exang', 'oldpeak', 'slope', 'ca', 'thal', 'predicted_heart_status']])


Cross-Validated Model Accuracy: 0.7000

Patient Predictions:
            age  sex  cp  trestbps    chol  fbs  restecg  thalch  exang  \
Patient CL   64    1   3    110.00  211.00    0        0  144.00      1   
Patient CK   63    1   3    140.00  463.00    0        2  104.00      0   
Patient CT   54    1   0     98.00  306.00    1        0  128.00      1   
Patient CJ   39    0   1    120.00  260.00    0        1  202.00      0   
Patient CD   77    1   1    356.00  145.00    0        1  143.00      0   
Patient CZ   45    0   3    102.00  333.00    0        2  117.00      1   
Patient CA   22    1   3    108.00  194.00    1        0  136.00      0   
Patient CF   51    0   2    190.00    0.00    0        0   92.00      0   
Patient CV   48    0   2    140.00  141.15    0        1   93.34      1   
Patient CM   56    0   1    135.82  142.00    1        0  226.17      1   

            oldpeak  slope  ca  thal  predicted_heart_status  
Patient CL    1.800      1   0     2              